In [2]:
import graph_tool.all as gt
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pyvis.network import Network


out_path = "output/results/"

# Create a color map
color_map = list(mcolors.TABLEAU_COLORS.values())
groups = ['2', '3']

# Function to pick a color from the color map
def pick_color(index):
    return color_map[index % len(color_map)]

/home/lschulte/VCS/cdbs_inducing_eval/env/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Prepare NetworkX Graph

In [3]:
df = pd.read_csv('input/relation_overview.csv', delimiter='\t')
df = df.rename(columns={'Group (base)': 'group_base'})
df['group_base'] = df['group_base'].astype(str)
# df = df[df['group_base'].isin(groups)]

df['A2_str'] = df['A2'].astype(str).apply(lambda x: x.replace('.0', ''))
df['A'] = df['A1'] + df['A2_str']
df['B2_str'] = df['B2'].astype(str).apply(lambda x: x.replace('.0', ''))
df['B'] = df['B1'] + df['B2_str']
source_groups = list(set(list(df['A1']) + list(df['B1'])))

nodes = list(df[['A', 'A1', 'A2', 'A2_str']].itertuples(index=False, name=None)) + list(df[['B', 'B1', 'B2', 'B2_str']].itertuples(index=False, name=None))
nodes = list(set(nodes))
nodes = [(n[0], {
    'label': n[0],
    'source': n[1], 
    'id': n[2],
    'source_id': n[3], 
    'color': pick_color(source_groups.index(n[1])), 
    'border_color': 'black',
    'shape': 'dot'
}) for n in nodes]

nodes.extend([
    (source, {
        'label': '', 
        'source': source, 
        'id': 0,
        'source_id': '', 
        'color': '#FF000000', 
        'border_color': '#FF000000',
        'shape': 'dot'
    }) for i, source in enumerate(source_groups)
])

nodes = sorted(nodes, key=lambda x: (x[1]['source'], x[1]['id']), reverse=False)

edges = []
for row in df.itertuples(index=False):

    if row.group_base in groups:
        edges.append((row.A, row.B, {
            'title': row.group_base,
            'A1': row.A1, 
            'A2': row.A2, 
            'B1': row.B1, 
            'B2': row.B2, 
            'color': pick_color(groups.index(row.group_base)), 
            'Method': row.Method
        }))



group_2_edges = [e for e in edges if e[2]['title'] == "2"]
group_3_edges = [e for e in edges if e[2]['title'] == "3"]

## Draw Interactive Graph

In [4]:
filename = f'{out_path}network_graph.html'

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.circular_layout(G)

# Plot with pyvis
net = Network(
    directed = False,
    select_menu = True, # Show part 1 in the plot (optional)
    filter_menu = True, # Show part 2 in the plot (optional)
    notebook=True,
    cdn_resources='in_line'
)

net.repulsion = 1000
net.show_buttons() # Show part 3 in the plot (optional)

net.from_nx(G) # Create directly from nx graph
net.toggle_physics(False)

for node, cords in pos.items():
    net.get_node(node)['x'] = cords[0] * 1000
    net.get_node(node)['y'] = cords[1] * 1000

net.write_html(filename)


In [5]:
len(group_2_edges), len(group_3_edges)

(69, 160)

## Draw PDF Graphs

In [ ]:
def draw_graph(group, group_tag, color='gray'):
    G = nx.Graph()
    G.add_nodes_from(reversed(nodes))
    G.add_edges_from(edges)

    plt.figure(figsize=(20, 20))
    pos = nx.circular_layout(G)

    node_colors = [properties['color'] for _, properties in reversed(nodes)]
    edge_colors = [properties['border_color'] for _, properties in reversed(nodes)]
    nx.draw_networkx_nodes(G, pos, node_size=1800, node_color=node_colors, edgecolors=edge_colors)

    labels = nx.get_node_attributes(G, 'source_id')

    nx.draw_networkx_labels(G, pos, labels, font_color='white', font_weight='heavy', font_size=20)

    # # Draw regular edges
    nx.draw_networkx_edges(G, pos, edgelist=group, edge_color=color)

    plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"{out_path}network_graph_group_{group_tag}.pdf")
    plt.title(f"Group {group_tag}")

draw_graph(group_2_edges, '2', pick_color(0))
draw_graph(group_3_edges, '3', pick_color(1))